In [1]:
import pandas as pd
import datetime
from datetime import datetime, timedelta
import os
import polars as pl
import gc
import numpy as np
from tqdm import tqdm
import pyarrow as pa
import pyarrow.parquet as pq

import s3fs
import boto3
from io import BytesIO as bo

In [2]:
#variables -
this_day = datetime.today()
### FOR TESTING - REMOVE LATER ### 
this_day = this_day - timedelta(days=7)
####
days_to_monday = (this_day.weekday() - 0) % 7
monday = this_day - timedelta(days=days_to_monday)

CUR_PROC_WK = monday.strftime("%Y%m%d")

PRE_PROC_WK0 = monday - timedelta(days=7)
PRE_PROC_WK = str(PRE_PROC_WK0.year) + str(PRE_PROC_WK0.month).zfill(2) + str(PRE_PROC_WK0.day).zfill(2)

CUR_WK0 = monday - timedelta(days=17)
CUR_WK = str(CUR_WK0.year) + str(CUR_WK0.month).zfill(2) + str(CUR_WK0.day).zfill(2)

PRE_WK0 = monday - timedelta(days=24)
PRE_WK = PRE_WK0.strftime("%Y%m%d")

In [3]:
bucket = 'vortex-staging-a65ced90'

In [4]:
#LIBNAMES
raw_path = f'PYADM/raw/{CUR_PROC_WK}/inbound/'
curwk = f'PYADM/raw/{CUR_PROC_WK}/dataframes/'
ptk = f'PYADM/weekly/archive/{CUR_WK}/plantrak/'

In [5]:
# Picking Up Data from Plantrak_2
# weekly - (NMLLAX stands for Normalized Lax)
#NMLLAX= pl.read_parquet(ptk+'\\LAX_N.parquet') #Source for this is subject to change , may add full version in future?
NMLLAX= pl.read_parquet(f's3://{bucket}/{ptk}LAX_N.parquet')

In [6]:
#Dropping rows where product is null
NMLLAX = NMLLAX.filter(pl.col('PROD_CD') != "") 
# This statement should be redundant as we kept inner join with seg def instead of left

In [7]:
#Picking up Date Parm from previous code to get week number and merging it rx data
#date_parm_wk = pl.read_parquet(curwk+"\\curwk_DATE_PARM_WK.parquet")
date_parm_wk = pl.read_parquet(f's3://{bucket}/{curwk}curwk_DATE_PARM_WK.parquet')
date_parm_wk = date_parm_wk.with_columns(pl.col('WK_END_DATE').dt.date()) 

#new fix-
NMLLAX = NMLLAX.with_columns(pl.col('WK_END_DATE').cast(pl.Date))
##

NMLLAX = NMLLAX.join(date_parm_wk,on='WK_END_DATE',how='left') # SHOULD I CHANGE THIS TO INNER ? 

In [8]:
metrics = ['TRX','NRX','TUN','NUN','TUF','NUF']
# Adding a new column called PROD_WK , will contain PROD_CD and the week number of transaction
# this the column on which we transpose the data  
NMLLAX = NMLLAX.with_columns([(pl.col("PROD_CD").cast(pl.Utf8) + "P_" + pl.col("I").cast(pl.Utf8)).alias("PROD_WK")])

# Only Keeping columns pertinent to LAX_DN
NMLLAX = NMLLAX.select(pl.col(['IID', 'PlanID', 'PlanName', 'PayerID', 'PayerName', 'PBMID' ,'PBMName', 'TRX', 'NRX','TUN', 'NUN', 'TUF','NUF','PROD_WK']))

# Sorting data at IID level to chunk and filter effectively
NMLLAX = NMLLAX.sort('IID')

#converting column to cat to save space -
NMLLAX = NMLLAX.with_columns(pl.col("PROD_WK").cast(pl.Utf8).cast(pl.Categorical))

In [9]:
# 'full_unique_vals' contains a list of all possible column names after transposing data
# we will use it to standardize the shape of every chunk, as it will help us concat / stack them each iteration

unique_vals = list(NMLLAX['PROD_WK'].unique()) #using full data to get all unique values here
# NOTE : It might very well be possible that some weeks of data may be missing, we might have to add those columns manually at some point

full_unique_vals = []
def unique_vals_prod_wk(col_name):   #this function breaks down PROD_WK to your regular column names like LI1PTUF
    parts = col_name.split('_')
    for m in metrics:
        full = parts[0]+m+parts[-1]
        full_unique_vals.append(full)
    
for i in unique_vals:
    unique_vals_prod_wk(i)

# Could add a modifier here to check and have full 105 weeks of data ?
    
full_unique_vals.sort()
full_unique_vals =['IID','PlanID', 'PlanName', 'PayerID', 'PayerName', 'PBMID' ,'PBMName'] + full_unique_vals
#Adding IID and others because i will also use this list to standardize the order of columns in each chunk

In [10]:
wk_tst = pl.DataFrame()
wk_tst = wk_tst.with_columns(pl.Series(name='col_names_raw',values=full_unique_vals[7:]))

def split_col_names(value):
    prod = value[:3]
    metric = value[4:7]
    wknum = value[7:]
    return prod, metric, wknum

wk_tst = wk_tst.with_columns([pl.col("col_names_raw").map_elements(split_col_names, return_dtype=pl.Object).alias("split_values")])

wk_tst = wk_tst.with_columns([
    pl.col("split_values").map_elements(lambda x: x[0], return_dtype=pl.Utf8).alias("prod"),
    pl.col("split_values").map_elements(lambda x: x[1], return_dtype=pl.Utf8).alias("metric"),
    pl.col("split_values").map_elements(lambda x: x[2], return_dtype=pl.Utf8).alias("wknum"),
])
wk_tst = wk_tst.drop(["split_values","col_names_raw"])

res = wk_tst.group_by(['prod','metric']).agg([pl.col('wknum').n_unique().alias('num_of_wks')])
missing_wknum = res.filter(pl.col('num_of_wks') != 105)
missing_wknum = missing_wknum.sort(by='prod')
print("Number of products in data which do not have 105 weeks of data : ",len(missing_wknum['prod'].unique()))
print(list((missing_wknum['prod'].unique())))
missmps = list((missing_wknum['prod'].unique()))
missing_wknum_print = missing_wknum.select(pl.col(['prod','num_of_wks']))
missing_wknum_print = missing_wknum_print.unique(subset=['prod','num_of_wks'])
print(missing_wknum_print)

wk_conti = pl.DataFrame()
wk_conti = wk_tst.filter(pl.col('prod').is_in(missmps))
wk_conti = wk_conti.drop('metric')
wk_conti = wk_conti.unique(subset=['prod','wknum'])
wk_conti = wk_conti.with_columns(pl.col("wknum").cast(pl.Int32))

print('BUT !  - ')
for prod in missmps:
    f1 = wk_conti.filter(pl.col('prod')== prod )
    if (len(f1['wknum'].unique()) != f1['wknum'].max()):
        print(prod," has gaps in weeks")
    

Number of products in data which do not have 105 weeks of data :  2
['MRB', 'ZEL']
shape: (2, 2)
┌──────┬────────────┐
│ prod ┆ num_of_wks │
│ ---  ┆ ---        │
│ str  ┆ u32        │
╞══════╪════════════╡
│ MRB  ┆ 83         │
│ ZEL  ┆ 55         │
└──────┴────────────┘
BUT !  - 
MRB  has gaps in weeks
ZEL  has gaps in weeks


# Transpose Data By 3000 HCP chunks-

In [11]:
unique_iids = NMLLAX['IID'].unique() 
chunk_size = 5000 #Each chunk will contain 5000 HCPs worth of transactions (NOT ROWS, they may differ each chunk)

iid_chunks = [unique_iids[i:i + chunk_size] for i in range(0, len(unique_iids), chunk_size)]
#So IID_chunks is a list of lists, each list contains 5000 HCPs and number of lists is our number of chunks

prod_family_market_buckets = {
    "MRXF" : ["MRGP","MRBP","GLYP"],
    "LINF" : ["LI1P","LI2P","LI3P"],
    "LUBF" : ["AMTP","LUBP"],
    "GENM" : ["FLXP","LACP","LUBP","MRGP","GLYP"],
    "BRDM" : ["AMTP","MRBP","LI1P","LI2P","LI3P","TRUP","MOTP","ZELP","IRLP"],
    "LAXM" : ["AMTP","FLXP","LACP","LUBP","MRGP","MRBP","LI1P","LI2P","LI3P","TRUP","GLYP","MOTP","ZELP","IRLP"]
}

writer = None
df_final = pl.DataFrame()
loop_counter = 0

In [12]:
for iid_chunk in tqdm(iid_chunks):
    
    # if loop_counter == 11:
    #     break
    
    df_chunk = NMLLAX.filter(pl.col('IID').is_in(iid_chunk))
    
    df_pivot_chunk = df_chunk.pivot(
        values=metrics,index=['IID','PlanID','PlanName','PayerID','PayerName','PBMID','PBMName'],
        columns='PROD_WK',
        maintain_order=True,
        sort_columns=True
    )
    
    del df_chunk
    gc.collect()
    
    df_pivot_chunk = df_pivot_chunk.select(
        pl.all().name.map(
            lambda col_name: col_name.split('_')[3] + col_name.split('_')[0] + col_name.split('_')[-1] if 'PROD_WK_' in col_name else col_name
        )
    )
    
    missing_cols = set(full_unique_vals) - set(df_pivot_chunk.columns)

    if missing_cols:
        df_missing = pl.DataFrame({col: pl.Series([None]*len(df_pivot_chunk), dtype=pl.Float64) for col in missing_cols})
        df_pivot_chunk = pl.concat([df_pivot_chunk, df_missing], how='horizontal')
    
    del df_missing
    gc.collect()

    df_pivot_chunk = df_pivot_chunk.select(full_unique_vals)
    
    for prod_family, prod_codes in prod_family_market_buckets.items():
        for metric in metrics:
            prod_metric_combinations = {prod_code + metric for prod_code in prod_codes}
            relevant_columns = [col for col in full_unique_vals if any(comb  in col for comb in prod_metric_combinations)]
            relevant_columns = [(col, col.split(metric)) for col in relevant_columns]
            week_numbers = sorted(set(int(parts[-1]) for col, parts in relevant_columns))
            for week_number in week_numbers:
                new_column = prod_family + metric + str(week_number)
                week_columns = [col for col, parts in relevant_columns if parts[-1] == str(week_number)]
                #df_pivot_chunk = df_pivot_chunk.with_columns(sum(pl.col(c) for c in week_columns).alias(new_column))
                df_pivot_chunk = df_pivot_chunk.with_columns(pl.sum_horizontal(week_columns).alias(new_column))

    df_final = df_final.vstack(df_pivot_chunk)
    
    loop_counter += 1

    if loop_counter % 5 == 0 and loop_counter != 0: # This takes about 25 seconds ? TTT Should be ~ 30 Secs
        table = df_final.to_arrow()
        if writer is None:
            #writer = pq.ParquetWriter(ptk+'\\LAX_DN.parquet', table.schema)
            writer = pq.ParquetWriter(f's3://{bucket}/{ptk}LAX_DN.parquet', table.schema)
        writer.write_table(table)
        del table
        gc.collect()
        df_final = pl.DataFrame() # Reset df_final after writing to file

    

# Write any remaining chunks to the Parquet file
if len(df_final) > 0:
    table = df_final.to_arrow()
    if writer is None:
        writer = pq.ParquetWriter(f's3://{bucket}/{ptk}LAX_DN.parquet', table.schema)
    writer.write_table(table)
    del table
    gc.collect()

# Close the ParquetWriter
if writer is not None:
    writer.close()

100%|██████████| 105/105 [44:40<00:00, 25.53s/it]


In [13]:
ptk

'PYADM/weekly/archive/20240531/plantrak/'

In [14]:
# unique_iids = NMLLAX['IID'].unique() 
# chunk_size = 3000
# iid_chunks = [unique_iids[i:i + chunk_size] for i in range(0, len(unique_iids), chunk_size)]

# writer = None
# df_final = pl.DataFrame()
# loop_counter = 0

In [15]:
# for iid_chunk in tqdm(iid_chunks): #4 is the max limit

#     if loop_counter == 5:
#         break
    
#     df_chunk = NMLLAX.filter(pl.col('IID').is_in(iid_chunk))
    
#     df_pivot_chunk = df_chunk.pivot(
#         values=metrics,index=['IID','PlanID','PlanName','PayerID','PayerName','PBMID','PBMName'],
#         columns='PROD_WK',
#         maintain_order=True,
#         sort_columns=True
#     )
    
#     df_pivot_chunk = df_pivot_chunk.select(
#         pl.all().name.map(
#             lambda col_name: col_name.split('_')[3] + col_name.split('_')[0] + col_name.split('_')[-1] if 'PROD_WK_' in col_name else col_name
#         )
#     )
    
#     del df_chunk
#     gc.collect()
    
#     missing_cols = set(full_unique_vals) - set(df_pivot_chunk.columns)

#     if missing_cols:
#         df_missing = pl.DataFrame({col: pl.Series([None]*len(df_pivot_chunk), dtype=pl.Float64) for col in missing_cols})
#         df_pivot_chunk = pl.concat([df_pivot_chunk, df_missing], how='horizontal')  
        
#     del df_missing
#     gc.collect()

#     df_pivot_chunk = df_pivot_chunk.select(full_unique_vals)

#     df_final = df_final.vstack(df_pivot_chunk)
    
#     loop_counter += 1

#     if loop_counter % 5 == 0 and loop_counter != 0: # This takes about 25 seconds ? TTT Should be ~ 30 Secs
#         table = df_final.to_arrow()
#         if writer is None:
#             writer = pq.ParquetWriter(ptk+'\\plan_df1.parquet', table.schema)
#         writer.write_table(table)
#         df_final = pl.DataFrame() # Reset df_final after writing to file

    

# # Write any remaining chunks to the Parquet file
# if len(df_final) > 0:
#     table = df_final.to_arrow()
#     if writer is None:
#         writer = pq.ParquetWriter(ptk+'\\plan_df1.parquet', table.schema)
#     writer.write_table(table)

# # Close the ParquetWriter
# if writer is not None:
#     writer.close()